In [1]:
import datetime
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from chart import *
from document import *

In [2]:
# 메타 정보

today = datetime.date.today()

y = today.year
m = today.month
d = today.day
w = today.weekday()

# report version
type = '월간'
month = '6월'
region = 'CN' # KR|CN|NA|EU|SG|RU
v = '1'

if region == 'KR':
    regex = '^(KR|Global|All)'
elif region == 'CN':
    regex = '^(CN|Global|All)'
elif region == 'NA':
    regex = '^(NA|Global|All)'
elif region == 'EU':
    regex = '^(EU|Global|All)'
elif region == 'SG':
    regex = '^(SG|Global|All)'
elif region == 'RU':
    regex = '^(SG|Global|All)'
else:
    regex = '^(KR|CN|NA|EU|SG|RU|Global|All)'

print("보고서 유형: ", type, "월: ", month)
print(regex)

보고서 유형:  월간 월:  6월
^(CN|Global|All)


## 보고서 표지 (커버 쉬트)

In [3]:
document.add_heading(f'[2022.06] {region} Cloud Infra 운영', level=0)
document.add_heading(f'{y}.{m}.{d}', level=1)
document.add_heading(f'클라우드운영센터', level=1)
document.add_page_break()

## 요약

In [4]:
# Document
document.add_paragraph('요약', style='Intense Quote')

In [5]:
# ETL
base_path = r"./data/요약_202206_서비스관리_0720_01.ods"
sheet = '6월집계'

df = read_ods(base_path, sheet, headers=True)

# 리전 & 운영계, 월간보고서용
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['월간']=='O')]

In [6]:
print(len(df), len(df_filtered))

56 3


In [7]:
df_filtered[df_filtered['월'] == '6월']

,년도,월,리전,AZ,테넌트,진행상태,성패,Description,주간,월간
54,22년,6월,CN,1센터,PRD,진행중,None,클라우드 플랫폼(opentsack) 보안 설정 현황 확인 요청\n중국 GCS시스템에...,O,O


In [8]:
# Chart
if region == 'All':
    # Pivoting
    pivot, total = getPivotTable(df)
    source = pivot
    chart = getPieChart(source)
    
    chart.save(f'./charts/summary_{y}_{m}_{d}_{v}_1.png')
    document.add_picture(f'./charts/summary_{y}_{m}_{d}_{v}_1.png')
else:
    pass

In [9]:
# Document
addSummaryTable(df_filtered, regex, month)
document.add_page_break()

## 1.1 모니터링(인시던트핸들링)

In [10]:
base_path = r"./data/1_1_202206_모니터링_인시던트핸들링_0720_01.ods"
sheet = '6월집계'

df = read_ods(base_path, sheet, headers=True)
df['장애전파'] = (pd.to_datetime(df['장애전파시간.1'])-pd.to_datetime(df['발생인지시간'])).astype('timedelta64[m]')

# Pivoting
pivot, total = getPivotTable(df)

# 리전 & 운영계, 월간보고서용
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료') & (df['월간']=='O')]
df_filtered, timeslot = getTables(df_filtered, regex, month)
    
# SLA failed
df_failed = df_filtered[ (df_filtered['장애전파'] > 10) & (df_filtered['월']== month)]
df_failed[['날짜', '장애대응조치내용', 'JIRA_Ticket_No', '장애전파']]

,날짜,장애대응조치내용,JIRA_Ticket_No,장애전파
92,2022.06.04,Deleted old snapshots,CNCLOUD2046,19.0
93,2022.06.08,Deleted old snapshots,None,30.0
100,2022.06.15,"Its known issue, it got resolved automatically",None,17.0


In [11]:
# Chart - Pie 차트
if region == 'All':
    source = pivot
    chart = getPieChart(source)
    chart.save(f'./charts/mon_{region}_{y}_{m}_{d}_1.png')
else:
    pass

# Chart - 스캐터 차트    
chart = getScatterChart(timeslot, month)
chart.save(f'./charts/mon_{region}_{y}_{m}_{d}_2.png')

# Chart - 라인 차트
chart = getLineChart(df_filtered, month)
chart.save(f'./charts/mon_{region}_{y}_{m}_{d}_3.png')

npm WARN config global `--global`, `--local` are deprecated. Use `--location=global` instead.
npm WARN config global `--global`, `--local` are deprecated. Use `--location=global` instead.
npm WARN config global `--global`, `--local` are deprecated. Use `--location=global` instead.


In [12]:
# Document
document.add_paragraph('모니터링(인시던트핸들링)', style='List Number')

if region == 'All':
    document.add_paragraph(f'6월: 총 {total}', style='List Bullet')    
    document.add_picture(f'./charts/mon_{region}_{y}_{m}_{d}_1.png')
else:
    pass

document.add_paragraph(f'장애전파 소요시간(분)', style='List Bullet')
document.add_picture(f'./charts/mon_{region}_{y}_{m}_{d}_2.png')
## 평균 소요 시간, 최대 소요 시간

document.add_paragraph(f'장애전파시간 초과 이벤트', style='List Bullet')
addFailedTable(df_failed, regex, month)
    
document.add_paragraph(f'월별 발생 건수 (추세)', style='List Bullet')
document.add_picture(f'./charts/mon_{region}_{y}_{m}_{d}_3.png')   

In [13]:
# Document
document.add_page_break()

## 1.2 이슈 관리

In [14]:
base_path = r"./data/1_2_202206_이슈관리_0720_01.ods"
sheet = '6월집계'

df = read_ods(base_path, sheet, headers=True)
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료') & (df['월간']=='O')]

# Pivoting
pivot, total = getPivotTable(df)

In [15]:
# Chart - Pie 차트
if region == 'All':
    source = pivot
    chart = getPieChart(source)
    chart.save(f'./charts/im_{region}_{y}_{m}_{d}_1.png')
else:
    pass

# Chart - 라인 차트
chart = getLineChart(df_filtered, month)
chart.save(f'./charts/im_{region}_{y}_{m}_{d}_2.png')

In [16]:
# Document
document.add_paragraph('이슈 관리', style='List Number')

if region == 'All':
    document.add_paragraph(f'6월: 총 {total}', style='List Bullet')    
    document.add_picture(f'./charts/im_{region}_{y}_{m}_{d}_1.png')
else:
    pass

document.add_paragraph(f'월별 발생 건수 (추세)', style='List Bullet')
document.add_picture(f'./charts/im_{region}_{y}_{m}_{d}_2.png')   

In [17]:
# Document
addSummaryTable(df_filtered, regex, month)
document.add_page_break()

## 1.3 장애 관리

In [18]:
base_path = r"./data/1_3_202206_장애관리_0720_01.ods"
sheet = '6월집계'

df = read_ods(base_path, sheet, headers=True)
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료') & (df['월간']=='O')]

# Pivoting
pivot, total = getPivotTable(df)

In [19]:
# Chart - Pie 차트
if region == 'All':
    source = pivot
    chart = getPieChart(source)
    chart.save(f'./charts/pm_{region}_{y}_{m}_{d}_1.png')
else:
    pass

# Chart - 라인 차트
chart = getLineChart(df_filtered, month)
chart.save(f'./charts/pm_{region}_{y}_{m}_{d}_2.png')

In [20]:
# Document
document.add_paragraph('장애(RCA) 관리', style='List Number')

if region == 'All':
    document.add_paragraph(f'6월: 총 {total}', style='List Bullet')    
    document.add_picture(f'./charts/pm_{region}_{y}_{m}_{d}_1.png')
else:
    pass

document.add_paragraph(f'월별 발생 건수 (추세)', style='List Bullet')
document.add_picture(f'./charts/pm_{region}_{y}_{m}_{d}_2.png')   

In [21]:
# Document
addSummaryTable(df_filtered, regex, month)
document.add_page_break()

## 1.4 변경 관리

In [22]:
base_path = r"./data/1_4_202206_변경관리_0720_01.ods"
sheet = '6월집계'

df = read_ods(base_path, sheet, headers=True)
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료') & (df['월간']=='O')]

# Pivoting
pivot, total = getPivotTable(df)

In [23]:
# Chart - Pie 차트
if region == 'All':
    source = pivot
    chart = getPieChart(source)
    chart.save(f'./charts/cm_{region}_{y}_{m}_{d}_1.png')
else:
    pass

# Chart - 라인 차트
chart = getLineChart(df_filtered, month)
chart.save(f'./charts/cm_{region}_{y}_{m}_{d}_2.png')

In [24]:
# Document
document.add_paragraph('변경 관리', style='List Number')

if region == 'All':
    document.add_paragraph(f'6월: 총 {total}', style='List Bullet')    
    document.add_picture(f'./charts/cm_{region}_{y}_{m}_{d}_1.png')
else:
    pass

document.add_paragraph(f'월별 발생 건수 (추세)', style='List Bullet')
document.add_picture(f'./charts/cm_{region}_{y}_{m}_{d}_2.png')   

In [25]:
# Document
addSummaryTable(df_filtered, regex, month)
document.add_page_break()

## 1.5 요청 관리

In [26]:
base_path = r"./data/1_5_202206_요청관리_0720_01.ods"
sheet = '6월집계'

df = read_ods(base_path, sheet, headers=True)
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료') & (df['월간']=='O')]

# Pivoting
pivot, total = getPivotTable(df)

In [27]:
# Chart - Pie 차트
if region == 'All':
    source = pivot
    chart = getPieChart(source)
    chart.save(f'./charts/cm_{region}_{y}_{m}_{d}_1.png')
else:
    pass

# Chart - 라인 차트
chart = getLineChart(df_filtered, month)
chart.save(f'./charts/cm_{region}_{y}_{m}_{d}_2.png')

In [28]:
# Document
document.add_paragraph('요청 관리', style='List Number')

if region == 'All':
    document.add_paragraph(f'6월: 총 {total}', style='List Bullet')    
    document.add_picture(f'./charts/cm_{region}_{y}_{m}_{d}_1.png')
else:
    pass

document.add_paragraph(f'월별 발생 건수 (추세)', style='List Bullet')
document.add_picture(f'./charts/cm_{region}_{y}_{m}_{d}_2.png')   

In [29]:
# Document
# addSummaryTable(df_filtered, regex, month)
document.add_page_break()

## 1.6 자산 관리 - VM

In [30]:
base_path = r"./data/1_6_202206_자산관리_VM_0720_01.ods"
sheet = '6월집계'

df = read_ods(base_path, sheet, headers=True)
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['월간']=='O')]

# Pivoting
pivot, total = getPivotTable(df)

In [31]:
# Chart - Pie 차트
if region == 'All':
    source = pivot
    chart = getPieChart2(df_filtered, month)
    chart.save(f'./charts/vm_{region}_{y}_{m}_{d}_1.png')
else:
    pass

# Chart - 라인 차트
chart = getLineChart2(df_filtered, month)
chart.save(f'./charts/vm_{region}_{y}_{m}_{d}_2.png')

In [32]:
# Document
document.add_paragraph('자산 관리(vm)', style='List Number')

if region == 'All':
    document.add_paragraph(f'6월: 총 {total}', style='List Bullet')    
    document.add_picture(f'./charts/vm_{region}_{y}_{m}_{d}_1.png')
else:
    pass

document.add_paragraph(f'월별 VM 수 (추세)', style='List Bullet')
document.add_picture(f'./charts/vm_{region}_{y}_{m}_{d}_2.png')   

In [33]:
# Document
# addSummaryTable(df_filtered, regex, month)
document.add_page_break()

## 1.6 자산 관리 - DB

In [34]:
base_path = r"./data/1_6_202206_자산관리_DB_0720_01.ods"
sheet = '6월집계'

df = read_ods(base_path, sheet, headers=True)
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['월간']=='O')]

# Pivoting
pivot, total = getPivotTable(df)

In [35]:
# Chart - Pie 차트
if region == 'All':
    source = pivot
    chart = getPieChart2(df_filtered, month)
    chart.save(f'./charts/db_{region}_{y}_{m}_{d}_1.png')
else:
    pass

# Chart - 라인 차트
chart = getLineChart2(df_filtered, month)
chart.save(f'./charts/db_{region}_{y}_{m}_{d}_2.png')

In [36]:
# Document
document.add_paragraph('자산 관리(db)', style='List Number')

if region == 'All':
    document.add_paragraph(f'6월: 총 {total}', style='List Bullet')    
    document.add_picture(f'./charts/db_{region}_{y}_{m}_{d}_1.png')
else:
    pass

document.add_paragraph(f'월별 추세', style='List Bullet')
document.add_picture(f'./charts/db_{region}_{y}_{m}_{d}_2.png')   

In [37]:
# Document
# addSummaryTable(df_filtered, regex, month)
document.add_page_break()

## 1.6 자산 관리 - K8s

In [38]:
base_path = r"./data/1_6_202206_자산관리_k8s_0720_01.ods"
sheet = '6월집계'

df = read_ods(base_path, sheet, headers=True)
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['월간']=='O')]

# Pivoting
pivot, total = getPivotTable(df)

In [39]:
# Chart - Pie 차트
if region == 'All':
    source = pivot
    chart = getPieChart(df_filtered, month)
    chart.save(f'./charts/k8s_{region}_{y}_{m}_{d}_1.png')
else:
    pass

# Chart - 라인 차트
chart = getLineChart(df_filtered, month)
chart.save(f'./charts/k8s_{region}_{y}_{m}_{d}_2.png')

In [40]:
# Document
document.add_paragraph('자산 관리(k8s)', style='List Number')

if region == 'All':
    document.add_paragraph(f'{month}: 총 {total}', style='List Bullet')    
    document.add_picture(f'./charts/k8s_{region}_{y}_{m}_{d}_1.png')
else:
    pass

document.add_paragraph(f'월별 추세', style='List Bullet')
document.add_picture(f'./charts/k8s_{region}_{y}_{m}_{d}_2.png')   

In [41]:
# Document
# addSummaryTable(df_filtered, regex, month)
document.add_page_break()

## 1.7 용량 관리

In [42]:
base_path = r"./data/1_7_202206_용량관리_0720_01.ods"
sheet = '6월집계'

df = read_ods(base_path, sheet, headers=True)
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['월간']=='O')]

# Pivoting
pivot, total = getPivotTable(df)

In [43]:
# Chart - Pie 차트
if region == 'All':
    source = pivot
    chart = getPieChart(df_filtered, month)
    chart.save(f'./charts/cpm_{region}_{y}_{m}_{d}_1.png')
else:
    pass

# Chart - 라인 차트
chart = getLineChart(df_filtered, month)
chart.save(f'./charts/cpm_{region}_{y}_{m}_{d}_2.png')

In [44]:
# Document
document.add_paragraph('용량(이슈) 관리', style='List Number')

if region == 'All':
    document.add_paragraph(f'{month}: 총 {total}', style='List Bullet')    
    document.add_picture(f'./charts/cpm_{region}_{y}_{m}_{d}_1.png')
else:
    pass

document.add_paragraph(f'월별 추세', style='List Bullet')
document.add_picture(f'./charts/cpm_{region}_{y}_{m}_{d}_2.png')   

In [45]:
# Document
addSummaryTable(df_filtered, regex, month)
document.add_page_break()

In [46]:
document.add_paragraph(f'자원 현황표', style='List Bullet')

## 1.8 백업 관리

In [47]:
base_path = r"./data/1_8_202206_백업관리_0720_01.ods"
sheet = '6월집계'

df = read_ods(base_path, sheet, headers=True)
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['월간']=='O')]

# Pivoting
pivot, total = getPivotTable(df)

In [48]:
# Chart - Pie 차트
if region == 'All':
    source = pivot
    chart = getPieChart(df_filtered, month)
    chart.save(f'./charts/bm_{region}_{y}_{m}_{d}_1.png')
else:
    pass

# Chart - 라인 차트
chart = getLineChart(df_filtered, month)
chart.save(f'./charts/bm_{region}_{y}_{m}_{d}_2.png')

In [49]:
# Document
document.add_paragraph('백업 관리', style='List Number')

if region == 'All':
    document.add_paragraph(f'{month}: 총 {total}', style='List Bullet')    
    document.add_picture(f'./charts/bm_{region}_{y}_{m}_{d}_1.png')
else:
    pass

document.add_paragraph(f'월별 추세', style='List Bullet')
document.add_picture(f'./charts/bm_{region}_{y}_{m}_{d}_2.png')   

In [50]:
# Document
# addSummaryTable(df_filtered, regex, month)
document.add_page_break()

## 1.9 보안 관리

In [51]:
base_path = r"./data/1_9_202206_보안관리_0720_01.ods"
sheet = '6월집계'

df = read_ods(base_path, sheet, headers=True)
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['월간']=='O')]

# Pivoting
pivot, total = getPivotTable(df)

In [ ]:
# Chart - Pie 차트
if region == 'All':
    source = pivot
    chart = getPieChart(df_filtered, month)
    chart.save(f'./charts/sm_{region}_{y}_{m}_{d}_1.png')
else:
    pass

# Chart - 라인 차트
chart = getLineChart(df_filtered, month)
chart.save(f'./charts/sm_{region}_{y}_{m}_{d}_2.png')

In [ ]:
# Document
document.add_paragraph('보안 관리', style='List Number')

if region == 'All':
    document.add_paragraph(f'{month}: 총 {total}', style='List Bullet')    
    document.add_picture(f'./charts/sm_{region}_{y}_{m}_{d}_1.png')
else:
    pass

document.add_paragraph(f'월별 추세', style='List Bullet')
document.add_picture(f'./charts/sm_{region}_{y}_{m}_{d}_2.png')   

In [ ]:
# Document
addSummaryTable(df_filtered, regex, month)
document.add_page_break()

## 1.10 하드웨어 정기점검

In [ ]:
base_path = r"./data/1_10_202206_정기점검_0720_01.ods"
sheet = '6월집계'

df = read_ods(base_path, sheet, headers=True)
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['월간']=='O')]

# Pivoting
pivot, total = getPivotTable(df)

In [ ]:
# Chart - Pie 차트
if region == 'All':
    source = pivot
    chart = getPieChart(df_filtered, month)
    chart.save(f'./charts/rc_{region}_{y}_{m}_{d}_1.png')
else:
    pass

# Chart - 라인 차트
chart = getLineChart(df_filtered, month)
chart.save(f'./charts/rc_{region}_{y}_{m}_{d}_2.png')

In [ ]:
# Document
document.add_paragraph('정기점검', style='List Number')

if region == 'All':
    document.add_paragraph(f'{month}: 총 {total}', style='List Bullet')    
    document.add_picture(f'./charts/rc_{region}_{y}_{m}_{d}_1.png')
else:
    pass

document.add_paragraph(f'월별 추세', style='List Bullet')
document.add_picture(f'./charts/rc_{region}_{y}_{m}_{d}_2.png')   

In [ ]:
# Document
# addSummaryTable(df_filtered, regex, month)
document.add_page_break()
document.save(f'{month}_{region}_월간보고서_{y}_{m}_{d}.docx')

In [ ]:
import datetime
from chart import *

# Data Loading
base_path = r"./data/1_7_202206_용량관리_0720_01.ods"
sheet = '6월집계'

df = read_ods(base_path, sheet, headers=True)
df

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
# Pivoting - [df['월']=='6월') & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
pivot, total = getPivotTable(df)
print(total)
pivot

In [ ]:
# ETL
total = Data.count()
data = getdata()
chart = getTable(data)

### 비중 :: Piot Table

In [ ]:
pivot = getPivotTable(df)

In [ ]:
source = pivot
chart = get_pieChart(source)
chart

In [ ]:
chart.save(f'./charts/mon_{y}_{m}_{d}_1.png')

In [ ]:
document.add_paragraph('1.1 서비스 관리', style='List Number')
document.add_paragraph(f'6월: 총 {total}', style='List Bullet')
document.add_picture(f'./charts/mon_{y}_{m}_{d}_1.png')

### 시계열 :: LineChart

In [ ]:
pTable

In [ ]:
regex = '^(KR|CN|NA|EU|SG|RU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

In [ ]:
chart.save('charts/2.png')
document.add_picture('charts/2.png')

In [ ]:
df_filtered[df_filtered['월'] == '6월'].sort_values(['리전']).reset_index()

### 시계열 :: BarChart

In [ ]:
source=pTable
chart = get_barChart(source)
chart

In [ ]:
chart.save('charts/3.png')
document.add_picture('charts/3.png')

### 시계열 - LineChart or BarChart

### ALL

In [ ]:
regex = '^(KR|CN|NA|EU|SG|RU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

In [ ]:
chart.save('charts/4.png')
document.add_picture('charts/4.png')

In [ ]:
source

In [ ]:
# Write to Word
document.add_paragraph('1.1 모니터링', style='List Number')
document.add_paragraph(f'6월: 총 {total}', style='List Bullet')
document.add_picture(f'./charts/mon_{y}_{m}_{d}_1.png')

### by Region

In [ ]:
regions = df['리전'].unique()

for region in regions:
    print(region)

#### KR

In [ ]:
regex = '^(KR|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

In [ ]:
chart.save('charts/5.png')
document.add_picture('charts/5.png')

#### CN

In [ ]:
regex = '^(CN|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

In [ ]:
chart.save('charts/6.png')
document.add_picture('charts/6.png')

#### NA

In [ ]:
regex = '^(NA|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

In [ ]:
chart.save('charts/7.png')
document.add_picture('charts/7.png')

#### EU

In [ ]:
regex = '^(EU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

In [ ]:
chart.save('charts/8.png')
document.add_picture('charts/8.png')

#### SG

In [ ]:
regex = '^(SG|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

In [ ]:
chart.save('charts/9.png')
document.add_picture('charts/9.png')

#### RU

In [ ]:
regex = '^(RU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

In [ ]:
chart.save('charts/10.png')
document.add_picture('charts/10.png')

In [ ]:
document.save('word.docx')

In [ ]:
import datetime
from chart import *

# Data Loading
base_path = r"./data/1_8_202206_백업관리_0720_01.ods"
sheet = '6월집계'

df = read_ods(base_path, sheet, headers=True)

In [ ]:
df

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
# Pivoting - [df['월']=='6월') & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
pivot, total = getPivotTable(df)
print(total)
pivot

In [ ]:
# ETL
total = Data.count()
data = getdata()
chart = getTable(data)

### 비중 :: Piot Table

In [ ]:
pivot = getPivotTable(df)

In [ ]:
source = pivot
chart = get_pieChart(source)
chart

In [ ]:
chart.save(f'./charts/mon_{y}_{m}_{d}_1.png')

In [ ]:
document.add_paragraph('1.1 서비스 관리', style='List Number')
document.add_paragraph(f'6월: 총 {total}', style='List Bullet')
document.add_picture(f'./charts/mon_{y}_{m}_{d}_1.png')

### 시계열 :: LineChart

In [ ]:
pTable

In [ ]:
regex = '^(KR|CN|NA|EU|SG|RU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

In [ ]:
chart.save('charts/2.png')
document.add_picture('charts/2.png')

In [ ]:
df_filtered[df_filtered['월'] == '6월'].sort_values(['리전']).reset_index()

### 시계열 :: BarChart

In [ ]:
source=pTable
chart = get_barChart(source)
chart

In [ ]:
chart.save('charts/3.png')
document.add_picture('charts/3.png')

### 시계열 - LineChart or BarChart

### ALL

In [ ]:
regex = '^(KR|CN|NA|EU|SG|RU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

In [ ]:
chart.save('charts/4.png')
document.add_picture('charts/4.png')

In [ ]:
source

In [ ]:
# Write to Word
document.add_paragraph('1.1 모니터링', style='List Number')
document.add_paragraph(f'6월: 총 {total}', style='List Bullet')
document.add_picture(f'./charts/mon_{y}_{m}_{d}_1.png')

### by Region

In [ ]:
regions = df['리전'].unique()

for region in regions:
    print(region)

#### KR

In [ ]:
regex = '^(KR|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

In [ ]:
chart.save('charts/5.png')
document.add_picture('charts/5.png')

#### CN

In [ ]:
regex = '^(CN|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

In [ ]:
chart.save('charts/6.png')
document.add_picture('charts/6.png')

#### NA

In [ ]:
regex = '^(NA|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

In [ ]:
chart.save('charts/7.png')
document.add_picture('charts/7.png')

#### EU

In [ ]:
regex = '^(EU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

In [ ]:
chart.save('charts/8.png')
document.add_picture('charts/8.png')

#### SG

In [ ]:
regex = '^(SG|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

In [ ]:
chart.save('charts/9.png')
document.add_picture('charts/9.png')

#### RU

In [ ]:
regex = '^(RU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

In [ ]:
chart.save('charts/10.png')
document.add_picture('charts/10.png')

In [ ]:
document.save('word.docx')

In [ ]:
# Data Loading
base_path = r"./data/1_9_202206_보안관리_0720_01.ods"
sheet = '6월집계'

df = read_ods(base_path, sheet, headers=True)

In [ ]:
df

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df.info()

In [ ]:
df.info()

In [ ]:
# Pivoting - [df['월']=='6월') & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
pivot, total = getPivotTable(df)
print(total)
pivot

In [ ]:
# ETL
total = Data.count()
data = getdata()
chart = getTable(data)

### 비중 :: Piot Table

In [ ]:
pivot = getPivotTable(df)

In [ ]:
source = pivot
chart = get_pieChart(source)
chart

In [ ]:
chart.save(f'./charts/mon_{y}_{m}_{d}_1.png')

In [ ]:
document.add_paragraph('1.1 서비스 관리', style='List Number')
document.add_paragraph(f'6월: 총 {total}', style='List Bullet')
document.add_picture(f'./charts/mon_{y}_{m}_{d}_1.png')

### 시계열 :: LineChart

In [ ]:
pTable

In [ ]:
regex = '^(KR|CN|NA|EU|SG|RU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

In [ ]:
chart.save('charts/2.png')
document.add_picture('charts/2.png')

In [ ]:
df_filtered[df_filtered['월'] == '6월'].sort_values(['리전']).reset_index()

### 시계열 :: BarChart

In [ ]:
source=pTable
chart = get_barChart(source)
chart

In [ ]:
chart.save('charts/3.png')
document.add_picture('charts/3.png')

### 시계열 - LineChart or BarChart

### ALL

In [ ]:
regex = '^(KR|CN|NA|EU|SG|RU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

In [ ]:
chart.save('charts/4.png')
document.add_picture('charts/4.png')

In [ ]:
source

In [ ]:
# Write to Word
document.add_paragraph('1.1 모니터링', style='List Number')
document.add_paragraph(f'6월: 총 {total}', style='List Bullet')
document.add_picture(f'./charts/mon_{y}_{m}_{d}_1.png')

### by Region

In [ ]:
regions = df['리전'].unique()

for region in regions:
    print(region)

#### KR

In [ ]:
regex = '^(KR|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

In [ ]:
chart.save('charts/5.png')
document.add_picture('charts/5.png')

#### CN

In [ ]:
regex = '^(CN|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

In [ ]:
chart.save('charts/6.png')
document.add_picture('charts/6.png')

#### NA

In [ ]:
regex = '^(NA|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

In [ ]:
chart.save('charts/7.png')
document.add_picture('charts/7.png')

#### EU

In [ ]:
regex = '^(EU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

In [ ]:
chart.save('charts/8.png')
document.add_picture('charts/8.png')

#### SG

In [ ]:
regex = '^(SG|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

In [ ]:
chart.save('charts/9.png')
document.add_picture('charts/9.png')

#### RU

In [ ]:
regex = '^(RU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

In [ ]:
chart.save('charts/10.png')
document.add_picture('charts/10.png')

In [ ]:
document.save('word.docx')

In [ ]:
# Data Loading
base_path = r"./data/1_10_202206_정기점검_0720_01.ods"
sheet = '6월집계'

df = read_ods(base_path, sheet, headers=True)

In [ ]:
df

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
# Pivoting - [df['월']=='6월') & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
pivot, total = getPivotTable(df)
print(total)
pivot

In [ ]:
# ETL
total = Data.count()
data = getdata()
chart = getTable(data)

### 비중 :: Piot Table

In [ ]:
pivot = getPivotTable(df)

In [ ]:
source = pivot
chart = get_pieChart(source)
chart

In [ ]:
chart.save(f'./charts/mon_{y}_{m}_{d}_1.png')

In [ ]:
document.add_paragraph('1.1 서비스 관리', style='List Number')
document.add_paragraph(f'6월: 총 {total}', style='List Bullet')
document.add_picture(f'./charts/mon_{y}_{m}_{d}_1.png')

### 시계열 :: LineChart

In [ ]:
pTable

In [ ]:
regex = '^(KR|CN|NA|EU|SG|RU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

In [ ]:
chart.save('charts/2.png')
document.add_picture('charts/2.png')

In [ ]:
df_filtered[df_filtered['월'] == '6월'].sort_values(['리전']).reset_index()

### 시계열 :: BarChart

In [ ]:
source=pTable
chart = get_barChart(source)
chart

In [ ]:
chart.save('charts/3.png')
document.add_picture('charts/3.png')

### 시계열 - LineChart or BarChart

### ALL

In [ ]:
regex = '^(KR|CN|NA|EU|SG|RU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

In [ ]:
chart.save('charts/4.png')
document.add_picture('charts/4.png')

In [ ]:
source

In [ ]:
# Write to Word
document.add_paragraph('1.1 모니터링', style='List Number')
document.add_paragraph(f'6월: 총 {total}', style='List Bullet')
document.add_picture(f'./charts/mon_{y}_{m}_{d}_1.png')

### by Region

In [ ]:
regions = df['리전'].unique()

for region in regions:
    print(region)

#### KR

In [ ]:
regex = '^(KR|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

In [ ]:
chart.save('charts/5.png')
document.add_picture('charts/5.png')

#### CN

In [ ]:
regex = '^(CN|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

In [ ]:
chart.save('charts/6.png')
document.add_picture('charts/6.png')

#### NA

In [ ]:
regex = '^(NA|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

In [ ]:
chart.save('charts/7.png')
document.add_picture('charts/7.png')

#### EU

In [ ]:
regex = '^(EU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

In [ ]:
chart.save('charts/8.png')
document.add_picture('charts/8.png')

#### SG

In [ ]:
regex = '^(SG|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

In [ ]:
chart.save('charts/9.png')
document.add_picture('charts/9.png')

#### RU

In [ ]:
regex = '^(RU|Global)'
df_filtered = df[df['리전'].apply(lambda x: True if re.search(regex, x) else False) & (df['테넌트']== 'PRD') & (df['진행상태']=='완료')]
source = df_filtered
chart = get_lineChart(source)
chart

In [ ]:
chart.save('charts/10.png')
document.add_picture('charts/10.png')

In [ ]:
document.save('word.docx')

In [ ]:
document.save(f'{month}_{region}_월간보고서_{y}_{m}_{d}.docx')